In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

24/11/03 03:35:36 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.3.0.66 instead (on interface wlp4s0)
24/11/03 03:35:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 03:35:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [18]:
sen_df = spark.createDataFrame([
    (0, 'Hi I head about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

In [19]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I head about S...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [20]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [21]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words',pattern= '\\W')

In [22]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [23]:
tokenized = tokenizer.transform(sen_df)

In [24]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I head about S...|[hi, i, head, abo...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [25]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I head about S...|[hi, i, head, abo...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [26]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [27]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I head about S...|[hi, i, head, abo...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [28]:
from pyspark.ml.feature import StopWordsRemover

In [32]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
     (1, ['Mary', 'had', 'a', 'little', 'lamb'])
], ['id', 'tokens'])

In [33]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [34]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [35]:
from pyspark.ml.feature import NGram

In [46]:
wordDataFrame = spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
    (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])
], ['id', 'words'])

In [47]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [48]:
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, java, c...|[I wish, wish jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [49]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [50]:
ngram = NGram(n=3, inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about Spark]                                  |
|[I wish java, wish java could, java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



In [51]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [72]:
sentenceData = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic,regression,models,are,neat')
], ['label', 'sentence'])

In [73]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [74]:
words_data = tokenizer.transform(sentenceData)

In [75]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |
+-----+-----------------------------------+------------------------------------------+



In [76]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [77]:
featurized_data = hashing_tf.transform(words_data)

In [78]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [79]:
idf_model = idf.fit(featurized_data)

In [80]:
rescaled_data = idf_model.transform(featurized_data)

In [81]:
rescaled_data.select('label', 'features').show(truncate=False)

24/11/03 04:13:22 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/11/03 04:13:22 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/11/03 04:13:23 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [82]:
from pyspark.ml.feature import CountVectorizer

In [83]:
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' '))
], ['id', 'words'])

In [84]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [85]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [86]:
model = cv.fit(df)

In [87]:
result = model.transform(df)

In [88]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

